## 행정동으로 변환

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [2]:
df = pd.read_csv('../data/final_DE.csv')
pd.set_option('display.max_columns',25)
df

,Unnamed: 0,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0
465,465,강동구,1174010500,길동,0,2,0,1,0,20,1,18,9,100,16,9,63,10,119,10,38,0


In [3]:
df.rename(columns={'Unnamed: 0':'index'},inplace=True)
df.set_index('index',inplace=True)
df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


### 행정동 기준

In [4]:
std = pd.read_excel('../data/한국행정구역분류_2022.10.1.기준_20221006123917.xlsx',sheet_name='행정동 및 법정동코드와의 연계표')
std

,한국행정구역분류와 행정동(행정기관) 및 법정동코드 연계표(2022.10.1.기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
1,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,11,1100000000,19880423,1100000000,NaN,Seoul,NaN
2,서울특별시,종로구,종로구,종로구,종로구,11010,11010,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
3,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720,1101072,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720,1101072,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,중문동,대포동,39020610,3902061,5013061000,20060701,5013011400,NaN,Jungmun-dong,NaN
21719,제주특별자치도,서귀포시,중문동,중문동,하원동,39020610,3902061,5013061000,20060701,5013011800,NaN,Jungmun-dong,NaN
21720,제주특별자치도,서귀포시,예래동,예래동,색달동,39020620,3902062,5013062000,20060701,5013011900,NaN,Yerae-dong,NaN
21721,제주특별자치도,서귀포시,예래동,예래동,상예동,39020620,3902062,5013062000,20060701,5013012000,NaN,Yerae-dong,NaN


In [5]:
std.columns = std.loc[0]

In [6]:
std = std[1:]
std

,시도,시군구,행정구역명,행정동\n(행정기관명),법정동,행정구역분류\n(개정_8자리),행정구역분류\n(7자리),행정기관코드,행정기관 생성일,법정동코드,법정동 관할구역\n분할여부,행정동 영문명칭,비고
1,서울특별시,서울특별시,서울특별시,서울특별시,서울특별시,11,11,1100000000,19880423,1100000000,NaN,Seoul,NaN
2,서울특별시,종로구,종로구,종로구,종로구,11010,11010,1111000000,19880423,1111000000,NaN,Jongno-gu,NaN
3,서울특별시,종로구,청운효자동,청운효자동,청운동,11010720,1101072,1111051500,20081101,1111010100,NaN,Cheongunhyoja-dong,NaN
4,서울특별시,종로구,청운효자동,청운효자동,신교동,11010720,1101072,1111051500,20081101,1111010200,NaN,Cheongunhyoja-dong,NaN
5,서울특별시,종로구,청운효자동,청운효자동,궁정동,11010720,1101072,1111051500,20081101,1111010300,NaN,Cheongunhyoja-dong,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21718,제주특별자치도,서귀포시,중문동,중문동,대포동,39020610,3902061,5013061000,20060701,5013011400,NaN,Jungmun-dong,NaN
21719,제주특별자치도,서귀포시,중문동,중문동,하원동,39020610,3902061,5013061000,20060701,5013011800,NaN,Jungmun-dong,NaN
21720,제주특별자치도,서귀포시,예래동,예래동,색달동,39020620,3902062,5013062000,20060701,5013011900,NaN,Yerae-dong,NaN
21721,제주특별자치도,서귀포시,예래동,예래동,상예동,39020620,3902062,5013062000,20060701,5013012000,NaN,Yerae-dong,NaN


In [7]:
std_seoul = std[std['시도'] == '서울특별시']

In [8]:
std_df = std_seoul[['시군구','행정동\n(행정기관명)','법정동','행정기관코드','법정동코드','법정동 관할구역\n분할여부']][1:]

In [9]:
std_df

,시군구,행정동\n(행정기관명),법정동,행정기관코드,법정동코드,법정동 관할구역\n분할여부
2,종로구,종로구,종로구,1111000000,1111000000,NaN
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [10]:
std_df.columns = ['시군구','행정동','법정동','행정기관코드','법정동코드','분할여부']
std_df

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
2,종로구,종로구,종로구,1111000000,1111000000,NaN
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [12]:
std_df[std_df['분할여부'] == '분할연계']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
12,종로구,청운효자동,세종로,1111051500,1111011900,분할연계
22,종로구,사직동,세종로,1111053000,1111011900,분할연계
48,종로구,종로1.2.3.4가동,세종로,1111061500,1111011900,분할연계
57,종로구,종로1.2.3.4가동,와룡동,1111061500,1111013000,분할연계
85,종로구,혜화동,와룡동,1111065000,1111013000,분할연계
...,...,...,...,...,...,...
762,강동구,성내제1동,성내동,1174064000,1174010800,분할연계
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


### 면적, 인구 데이터

In [13]:
area_df = pd.read_csv('../data/AREA_DONG.csv')
del(area_df['SI_DO'])
del(area_df['GU'])

In [14]:
area_df

,DONG,AREA
0,청운동,"797,032.5"
1,신교동,"99,144.0"
2,궁정동,"73,042.7"
3,효자동,"59,135.9"
4,창성동,"59,706.1"
...,...,...
462,둔촌동,"2,461,511.8"
463,암사동,"4,711,364.2"
464,성내동,"1,958,987.6"
465,천호동,"3,065,543.3"


In [15]:
population_df = pd.read_csv('../data/POPU_DONG.csv')
del(population_df['DONG_CODE'])
del(population_df['SI_DO'])
del(population_df['GU'])

In [16]:
population_df

,DONG,NUM_POPU
0,청운동,2330
1,신교동,1393
2,궁정동,179
3,효자동,572
4,창성동,439
...,...,...
462,둔촌동,24311
463,암사동,66490
464,성내동,65873
465,천호동,85928


In [17]:
population_df['idx'] = [i for i in range(467)]
population_df.set_index('idx',inplace=True)

In [18]:
area_df['idx'] = [i for i in range(467)]
area_df.set_index('idx',inplace=True)

In [19]:
area_df

,DONG,AREA
idx,,
0,청운동,"797,032.5"
1,신교동,"99,144.0"
2,궁정동,"73,042.7"
3,효자동,"59,135.9"
4,창성동,"59,706.1"
...,...,...
462,둔촌동,"2,461,511.8"
463,암사동,"4,711,364.2"
464,성내동,"1,958,987.6"


In [20]:
density_df = pd.merge(area_df,population_df,how='outer',on='idx')

In [21]:
density_df

,DONG_x,AREA,DONG_y,NUM_POPU
idx,,,,
0,청운동,"797,032.5",청운동,2330
1,신교동,"99,144.0",신교동,1393
2,궁정동,"73,042.7",궁정동,179
3,효자동,"59,135.9",효자동,572
4,창성동,"59,706.1",창성동,439
...,...,...,...,...
462,둔촌동,"2,461,511.8",둔촌동,24311
463,암사동,"4,711,364.2",암사동,66490
464,성내동,"1,958,987.6",성내동,65873


In [33]:
l = []
for i in range(467):
    if(density_df['DONG_x'][i] == density_df['DONG_y'][i]):
        l.append(0)
    else:
        l.append(1)

In [35]:
from collections import Counter
Counter(l)

Counter({0: 467})

In [36]:
del(density_df['DONG_y'])

In [37]:
density_df

,DONG_x,AREA,NUM_POPU
idx,,,
0,청운동,"797,032.5",2330
1,신교동,"99,144.0",1393
2,궁정동,"73,042.7",179
3,효자동,"59,135.9",572
4,창성동,"59,706.1",439
...,...,...,...
462,둔촌동,"2,461,511.8",24311
463,암사동,"4,711,364.2",66490
464,성내동,"1,958,987.6",65873


In [40]:
density_df.columns = ['법정동','면적','인구']
density_df

,법정동,면적,인구
idx,,,
0,청운동,"797,032.5",2330
1,신교동,"99,144.0",1393
2,궁정동,"73,042.7",179
3,효자동,"59,135.9",572
4,창성동,"59,706.1",439
...,...,...,...
462,둔촌동,"2,461,511.8",24311
463,암사동,"4,711,364.2",66490
464,성내동,"1,958,987.6",65873


In [47]:
area = density_df['면적'].tolist()
population = density_df['인구'].tolist()
for i in range(len(area)):
    area[i] = float(area[i].replace(',',''))
density = []
for i in range(len(density_df.index)):
    dense = population[i] / area[i]
    density.append(dense)
density_df['인구밀도'] = density

In [48]:
density_df

,법정동,면적,인구,인구밀도
idx,,,,
0,청운동,"797,032.5",2330,0.002923
1,신교동,"99,144.0",1393,0.014050
2,궁정동,"73,042.7",179,0.002451
3,효자동,"59,135.9",572,0.009673
4,창성동,"59,706.1",439,0.007353
...,...,...,...,...
462,둔촌동,"2,461,511.8",24311,0.009876
463,암사동,"4,711,364.2",66490,0.014113
464,성내동,"1,958,987.6",65873,0.033626


In [49]:
std_df

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
2,종로구,종로구,종로구,1111000000,1111000000,NaN
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [50]:
df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [53]:
from IPython.display import display, HTML

display(
    HTML(
        data="""
                <style>
                    div#notebook-container    { width: 70%; }
                    div#menubar-container     { width: 70%; }
                    div#maintoolbar-container { width: 99%; }
                </style>
        """
    )
)

In [60]:
std_df = std_df[std_df['시군구'] != std_df['행정동']]
std_df

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
7,종로구,청운효자동,창성동,1111051500,1111010500,NaN
...,...,...,...,...,...,...
763,강동구,성내제2동,성내동,1174065000,1174010800,분할연계
764,강동구,성내제3동,성내동,1174066000,1174010800,분할연계
765,강동구,길동,길동,1174068500,1174010500,NaN
766,강동구,둔촌제1동,둔촌동,1174069000,1174010600,분할연계


In [61]:
std_df[std_df['분할여부'] != '분할연계']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
7,종로구,청운효자동,창성동,1111051500,1111010500,NaN
...,...,...,...,...,...,...
727,송파구,오금동,오금동,1171057000,1171011200,NaN
730,송파구,석촌동,석촌동,1171060000,1171010500,NaN
731,송파구,삼전동,삼전동,1171061000,1171010600,NaN
749,강동구,강일동,강일동,1174051500,1174011000,NaN


In [62]:
df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [182]:
tmp = std_df[std_df['행정동'] == '청운효자동']
tmp

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
3,종로구,청운효자동,청운동,1111051500,1111010100,NaN
4,종로구,청운효자동,신교동,1111051500,1111010200,NaN
5,종로구,청운효자동,궁정동,1111051500,1111010300,NaN
6,종로구,청운효자동,효자동,1111051500,1111010400,NaN
7,종로구,청운효자동,창성동,1111051500,1111010500,NaN
8,종로구,청운효자동,통인동,1111051500,1111010800,NaN
9,종로구,청운효자동,누상동,1111051500,1111010900,NaN
10,종로구,청운효자동,누하동,1111051500,1111011000,NaN
11,종로구,청운효자동,옥인동,1111051500,1111011100,NaN
12,종로구,청운효자동,세종로,1111051500,1111011900,분할연계


In [183]:
y = []
for x in df['DONG'].tolist():
    for i in range(len(tmp['법정동'])):
        if(x == tmp['법정동'].tolist()[i]):
            y.append(x)
y

['청운동', '신교동', '궁정동', '효자동', '창성동', '통인동', '누상동', '누하동', '옥인동', '세종로']

In [184]:
z = pd.DataFrame(columns=df.columns)
for i in range(len(y)):
    a = df[df['DONG'] == y[i]]
    z = pd.concat([z,a])
z

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
5,종로구,1111010800,통인동,0,0,0,0,0,0,0,0,0,5,2,0,4,0,0,0,0,0
6,종로구,1111010900,누상동,0,0,0,0,0,0,0,0,0,1,0,0,1,0,10,0,0,0
7,종로구,1111011000,누하동,0,0,0,0,0,0,0,0,0,2,0,0,8,0,3,2,0,0
8,종로구,1111011100,옥인동,0,0,0,1,0,0,0,0,0,2,0,0,5,0,15,3,0,0
9,종로구,1111011900,세종로,1,1,0,0,0,0,0,0,0,0,0,5,11,3,8,1,0,0


In [107]:
m = 0
for i in range(len(z.index)):
    m += z['SUBWAY_NUM'][i]
m
# z.loc[len(z.index)] = 
# len(z.index)

1

In [185]:
n = ['종로구',1111000000,'청운효자동']
for c in z.columns[3:]:
    n.append(sum(z[c]))
n

['종로구',
 1111000000,
 '청운효자동',
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 18,
 4,
 9,
 47,
 3,
 62,
 8,
 2,
 0]

In [187]:
sample_df = pd.DataFrame(columns=df.columns)
sample_df.loc[0] = n
sample_df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
0,종로구,1111000000,청운효자동,1,1,0,1,0,0,0,0,0,18,4,9,47,3,62,8,2,0


In [95]:
std_df[std_df['법정동'] == '세종로']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
12,종로구,청운효자동,세종로,1111051500,1111011900,분할연계
22,종로구,사직동,세종로,1111053000,1111011900,분할연계
48,종로구,종로1.2.3.4가동,세종로,1111061500,1111011900,분할연계


In [96]:
df[df['DONG'] == '세종로']

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
9,종로구,1111011900,세종로,1,1,0,0,0,0,0,0,0,0,0,5,11,3,8,1,0,0


In [100]:
std_df[std_df['분할여부'] == '분할연계']['법정동'].unique()

array(['세종로', '와룡동', '창신동', '숭인동', '남대문로2가', '남대문로3가', '남대문로4가', '봉래동1가',
       '순화동', '회현동1가', '회현동2가', '충무로1가', '예장동', '충무로4가', '충무로5가', '묵정동',
       '장충동2가', '신당동', '이촌동', '이태원동', '하왕십리동', '행당동', '성수동1가', '성수동2가',
       '송정동', '중곡동', '자양동', '구의동', '전농동', '답십리동', '장안동', '휘경동', '이문동',
       '면목동', '상봉동', '중화동', '묵동', '망우동', '신내동', '돈암동', '정릉동', '길음동',
       '하월곡동', '장위동', '미아동', '번동', '수유동', '창동', '도봉동', '쌍문동', '방학동',
       '월계동', '공릉동', '하계동', '중계동', '상계동', '불광동', '갈현동', '응암동', '신사동',
       '북아현동', '홍제동', '홍은동', '남가좌동', '북가좌동', '아현동', '공덕동', '염리동', '도화동',
       '마포동', '대흥동', '노고산동', '신수동', '신정동', '망원동', '성산동', '목동', '신월동',
       '가양동', '등촌동', '화곡동', '마곡동', '내발산동', '공항동', '방화동', '구로동', '고척동',
       '개봉동', '오류동', '독산동', '시흥동', '영등포동', '신길동', '양평동3가', '대림동', '노량진동',
       '상도동', '사당동', '신대방동', '봉천동', '신림동', '서초동', '방배동', '잠원동', '반포동',
       '양재동', '압구정동', '논현동', '삼성동', '대치동', '역삼동', '도곡동', '개포동', '일원동',
       '수서동', '풍납동', '거여동', '마천동', '방이동', '송파동', '가락동', '문정동', '장지동',
  

In [103]:
density_df[density_df['법정동'] == '세종로']

,법정동,면적,인구,인구밀도
idx,,,,
18,세종로,"888,697.3",0,0.0


In [104]:
std_df[std_df['법정동'] == '와룡동']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
57,종로구,종로1.2.3.4가동,와룡동,1111061500,1111013000,분할연계
85,종로구,혜화동,와룡동,1111065000,1111013000,분할연계


In [105]:
std_df[std_df['법정동'] == '창신동']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
91,종로구,창신제1동,창신동,1111067000,1111017400,분할연계
92,종로구,창신제2동,창신동,1111068000,1111017400,분할연계
93,종로구,창신제3동,창신동,1111069000,1111017400,분할연계


In [106]:
std_df[std_df['법정동'] == '숭인동']

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
94,종로구,숭인제1동,숭인동,1111070000,1111017500,분할연계
95,종로구,숭인제2동,숭인동,1111071000,1111017500,분할연계


In [121]:
tmp_df = df.copy()
tmp_df

,GU,DONG_CODE,DONG,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,,,,
0,종로구,1111010100,청운동,0,0,0,0,0,0,0,0,0,2,0,2,8,0,12,0,1,0
1,종로구,1111010200,신교동,0,0,0,0,0,0,0,0,0,5,2,0,1,0,14,0,0,0
2,종로구,1111010300,궁정동,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,종로구,1111010400,효자동,0,0,0,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0
4,종로구,1111010500,창성동,0,0,0,0,0,0,0,0,0,1,0,1,3,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,강동구,1174010700,암사동,1,1,0,2,0,30,4,16,8,197,10,11,70,4,133,11,33,0
463,강동구,1174010900,천호동,1,4,0,3,1,70,6,14,7,124,8,15,131,17,266,12,68,1
464,강동구,1174010800,성내동,4,4,0,2,0,50,2,30,14,190,16,18,143,20,156,12,58,0


In [140]:
std_df[(std_df['분할여부'] == '분할연계')&(std_df['법정동']=='신정동')]

,시군구,행정동,법정동,행정기관코드,법정동코드,분할여부
474,마포구,신수동,신정동,1144063000,1144011700,분할연계
478,마포구,서강동,신정동,1144065500,1144011700,분할연계
504,양천구,신정1동,신정동,1147062000,1147010100,분할연계
505,양천구,신정2동,신정동,1147063000,1147010100,분할연계
506,양천구,신정3동,신정동,1147064000,1147010100,분할연계
507,양천구,신정4동,신정동,1147065000,1147010100,분할연계
508,양천구,신정6동,신정동,1147067000,1147010100,분할연계
509,양천구,신정7동,신정동,1147068000,1147010100,분할연계


In [143]:
o = std_df[std_df['분할여부'] == '분할연계']['법정동코드'].value_counts()
o = pd.DataFrame(o)
o

,법정동코드
1162010200,11
1162010100,9
1135010500,8
1150010300,8
1156013200,7
...,...
1150010600,2
1123011000,2
1126010200,2
1126010300,2


In [158]:
for x in o.index:
    if(tmp_df['DONG_CODE'] == x):
        tp = tmp_df[tmp_df['DONG_CODE'] == o.index[3]]
        tp[tp.columns[3:]] /= o[o.index == x]['법정동코드']
tp

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [171]:
lis = []
for x in o.index:
    k = o[o.index == x]['법정동코드'].tolist() # 해당 법정동코드에 속하는 법정동 수
    lis.append(k)
lis

[[11],
 [9],
 [8],
 [8],
 [7],
 [7],
 [6],
 [6],
 [6],
 [6],
 [6],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [5],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [4],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [3],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2]]

In [165]:
tp = tmp_df[tmp_df['DONG_CODE'] == o.index[3]]
b = tp[tp.columns[3:]]

In [167]:
b

,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,
352,1,6,0,9,1,160,5,56,16,341,36,28,263,38,321,175,104,0


In [168]:
for i in range(len(b.columns)):
    b[b.columns[i]] = b[b.columns[i]] / k.to_list()
b

,SUBWAY_NUM,STARBUCKS_NUM,SPORT_NUM,SAFE_DLVR_NUM,MC_NUM,KINDER_NUM,KIDS_NUM,GYM_NUM,GOLF_NUM,ACADEMY_NUM,ANIMAL_HSPT_NUM,BIKE_NUM,CAFE,CAR_SHR_NUM,CCTV_NUM,CHILD_MED_NUM,CON_NUM,DPTM_NUM
index,,,,,,,,,,,,,,,,,,
352,0.5,3.0,0.0,4.5,0.5,80.0,2.5,28.0,8.0,170.5,18.0,14.0,131.5,19.0,160.5,87.5,52.0,0.0


In [181]:
x = pd.DataFrame()
for i in range(len(o.index)):
    tp = tmp_df[tmp_df['DONG_CODE'] == o.index[i]]
    pd.concat(x,tp)


ValueError: cannot set a row with mismatched columns

In [ ]:
o = pd.DataFrame(std_df[std_df['분할여부'] == '분할연계']['법정동코드'].value_counts())

lis = []
for x in o.index:
    k = o[o.index == x]['법정동코드'].tolist() # 해당 법정동코드에 속하는 법정동 수
    lis.append(k)
lis

tp = tmp_df[tmp_df['DONG_CODE'] == o.index[3]]
b = tp[tp.columns[3:]]
for i in range(len(b.columns)):
    b[b.columns[i]] = b[b.columns[i]] / k.to_list()
b